# Exploratory Data Analysis for Tuesday, October 24
  

The goal for this notebook is: 

1. Basic Data Wrangling

Follow along as we work our way through this document. Use the code cells to test and experiment as we go. Try to complete all of the practice exercises and ask questions if you need assistance.



## Indexing

Pandas allows users to have more than one index levels on an axis. Think of this feature as a way to flatten out higher dimensional data - like a pivot table in Excel. Suppose, for instance, you needed to track data about Minnesota and Wisconsin from 2022 and 2023:

In [2]:
import pandas as pd
index = [('Minnesota', 2022), ('Minnesota', 2023),
         ('Wisconsin', 2022), ('Wisconsin', 2023),
         ('Iowa', 2022), ('Iowa', 2023)]
gdp = [371648, 372556,
               189757, 193702,
               208820, 255561]
output = pd.Series(gdp, index=index)
output

(Minnesota, 2022)    371648
(Minnesota, 2023)    372556
(Wisconsin, 2022)    189757
(Wisconsin, 2023)    193702
(Iowa, 2022)         208820
(Iowa, 2023)         255561
dtype: int64

The code above uses Python tuples as keys. This indexing scheme allows you to index or slice the series based on this multiple index:

In [3]:
output[('Wisconsin', 2023):('Iowa', 2022)]

(Wisconsin, 2023)    193702
(Iowa, 2022)         208820
dtype: int64

As you will find, however, this is an inconvenient way to index your data. For example, if you need to select all values from 2023 the code will be ugly and overly complex:

In [4]:
output[[i for i in output.index if i[1] == 2023]]


(Minnesota, 2023)    372556
(Wisconsin, 2023)    193702
(Iowa, 2023)         255561
dtype: int64

Pandas provides a better solution: the Pandas MultiIndex. This type allows you to create a multi-index from the tuples as follows:

In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('Minnesota', 2022),
            ('Minnesota', 2023),
            ('Wisconsin', 2022),
            ('Wisconsin', 2023),
            (     'Iowa', 2022),
            (     'Iowa', 2023)],
           )

See how the MultiIndex contains multiple levels of indexing–in? Re-indexing the output series with this MultiIndex shows a hierarchical representation of the data:

In [6]:
output = output.reindex(index)
output

Minnesota  2022    371648
           2023    372556
Wisconsin  2022    189757
           2023    193702
Iowa       2022    208820
           2023    255561
dtype: int64

The first two columns of the Series representation show the multiple index values. The third column shows the data. See how some entries are missing in the first column? In this multi-index representation blank entries indicate the same value as the line above it.

To access all data for which the second index is 2023 you can use the Pandas slicing notation:

In [7]:
output[:, 2023]

Minnesota    372556
Wisconsin    193702
Iowa         255561
dtype: int64

In [8]:
output["Wisconsin", :]

2022    189757
2023    193702
dtype: int64

We are left with a clean, single indexed array. This syntax is more efficient than the tuple-based multi-indexing solution that we initially tried. Hierarchical indexing can be used to rearrange our data into a DataFrame using the unstack method: 

In [9]:
output.unstack()

,2022,2023
Iowa,208820,255561
Minnesota,371648,372556
Wisconsin,189757,193702


The inverse can be achieved using the stack operation

In [10]:
output.unstack().stack()

Iowa       2022    208820
           2023    255561
Minnesota  2022    371648
           2023    372556
Wisconsin  2022    189757
           2023    193702
dtype: int64

We can also add a hierarchical index to the columns axis.

In [11]:
import numpy as np
data = pd.DataFrame(np.arange(16).reshape((4,4)),
                    index =[['Iowa', 'Iowa', 'Wisconsin','Wisconsin'], ['1','2','1','2']],
                    columns=[['2022', '2022', '2023', '2023'],
                            ['Dec.', 'Jan.', 'Dec.', 'Jan.']])
data

2022      2023     
            Dec. Jan. Dec. Jan.
Iowa      1    0    1    2    3
          2    4    5    6    7
Wisconsin 1    8    9   10   11
          2   12   13   14   15

## Reordering and Sorting Levels

Sometimes we will want to rearrange the order of the levels on an axis. This can be achieved using the `swaplevel` method. 

In [12]:
sort = pd.DataFrame(np.arange(16).reshape((4,4)),
                    index =[['X', 'X', 'Y','Y'], ['1','2','1','2']],
                    columns=[['2022', '2022', '2023', '2023'],
                            ['Dec.', 'Jan.', 'Dec.', 'Jan.']])

In [13]:
sort

2022      2023     
    Dec. Jan. Dec. Jan.
X 1    0    1    2    3
  2    4    5    6    7
Y 1    8    9   10   11
  2   12   13   14   15

In [14]:
sort.index.names = ['name1','name2']

In [15]:
sort.swaplevel('name1','name2')

2022      2023     
            Dec. Jan. Dec. Jan.
name2 name1                    
1     X        0    1    2    3
2     X        4    5    6    7
1     Y        8    9   10   11
2     Y       12   13   14   15

Another useful tool is the `sort_index` method, which sorts data using only hthe values in a single level. 

In [16]:
sort.sort_index(level=0)

2022      2023     
            Dec. Jan. Dec. Jan.
name1 name2                    
X     1        0    1    2    3
      2        4    5    6    7
Y     1        8    9   10   11
      2       12   13   14   15

In [17]:
sort.sort_index(level=1)

2022      2023     
            Dec. Jan. Dec. Jan.
name1 name2                    
X     1        0    1    2    3
Y     1        8    9   10   11
X     2        4    5    6    7
Y     2       12   13   14   15

## Combining and Merging Datasets


It is rare that you will not need to merge datasets together for a data analysis project. This is achieved by matching up the two datasets using one or more variables as an index. 

Let's use the auto dataset to create a contrived example.

In [25]:
# The data is saved as a csv file. Create two new dataframes with part of the data in each
auto = pd.read_csv('../data/auto.csv')
auto1 = auto[['name', 'horsepower',]]
auto2 = auto[['name', 'cylinders',]]

In [20]:
# The data is saved as a csv file. Create two new dataframes with part of the data in each
auto1 = auto[auto['origin']==1]
auto2.shape

(392, 2)

In [21]:
auto2 = auto[auto['origin']==2]

Let's now try to create one DataFrame with cylinders and horsepower as columns. 

### Merge the two DataFrames into one DataFrame
We want to match the two DataFrames together according to the name. 
We refer to this variable we are matching on as the **key** So, in our case, the key is the vehicle name

On more complicated merges we will need to tell pandas how to treat keys that are not present in both databases. The different types of 'join' (more database-ese) are
1. **inner**: keep the intersection of the keys
2. **left**: keep all the keys from the left DataFrame
3. **right**: keep all the keys from right DataFrame
4. **outer**: keep all the keys from both DataFrames

We specify the join type with the `how` parameter. The default is inner, but be explicit about your join.

In [26]:
# left and right specify the DataFrames to merge, on specifies the key
autos = pd.merge(left=auto1, right=auto2, on=['name'], how='outer')
autos

,name,horsepower,cylinders
0,chevrolet chevelle malibu,130,8
1,chevrolet chevelle malibu,130,6
2,chevrolet chevelle malibu,100,8
3,chevrolet chevelle malibu,100,6
4,buick skylark 320,165,8
...,...,...,...
671,ford mustang gl,86,4
672,vw pickup,52,4
673,dodge rampage,84,4
674,ford ranger,79,4
